In [1]:
### Connect using gspread

import gspread

gc = gspread.service_account(filename = 'Babysleep-tryout-client-secret2.json')

# get the instance of the Spreadsheet
sheet = gc.open('Babysleep-diaries_18.01.21-19.02.21')

In [2]:
import regex as re
import pandas as pd
import gspread_dataframe as gd

# prepare a list with all sheet names from the spreadsheet and select the part of the statement,
# that function worksheets writes, that contains only the actual sheet name

sheet_names = []
for name in sheet.worksheets():
    #print(str(name))
    sheet_names.append(re.findall(r"'(.*?)'", str(name))[0])
#print(sheet_names)

dict_df = {}
for name in sheet_names:
    sheet_instance = sheet.worksheet(name)
    dict_df[name] = gd.get_as_dataframe(sheet_instance,evaluate_formulas = True,skiprows = 1, has_header = True)
    print("done for sheet {}".format(name))

done for sheet 1. Иванова Анна (Яна)
done for sheet 2. Петрова Алена (Вероника)
done for sheet 3. Сидорова Анастасия (Нина)
done for sheet 4. Николаевы Варвара и Антон (Ия)
done for sheet 5. Шевченко Наталья (Демьян)


In [ ]:
# How to write columns from one DF to another, if they are both in the dictionaries

"""

dict_col1 = {}

columns1 = ['Дата ', 
            'УТРО (проснулись)', 
            'ВРЕМЯ засыпания в НОЧЬ (пережолили / отпустил грудь / вышли из комнаты и т.п.)', 
            'Кол-во ночных проб-й']
columns2 = ['Date', 
            'First wake up time', 
            'Sleep starting time', 
            'Nr of night wake ups']

for key, value in dict_df.items():
    dict_col1[key] = value.loc[:,columns1]
    print(dict_col1[key].shape)

dict_col2 = {}
for name in sheet_names:
    dict_col2[name] = dict_df[name].loc[:,columns1]
    dict_col2[name].columns = columns2
    print(dict_col2[name].columns)

In [3]:
def create_dc(dc, list1, list2):
    """ Creates a new dictionary with DataFrames from an existing dictionary dc.
    From df1 data from columns from list1 will be copied. The columns in the new DataFrame
    wil be called according to the list2. The order of the names must be the same in list1 and list2.
    """
    dc2 = {}
    for key, value in dc.items():
        dc2[key] = value.loc[:, list1]
        dc2[key].columns = list2
    return dc2

columns1 = ['Дата ', 
            'УТРО (проснулись)', 
            'ВРЕМЯ засыпания в НОЧЬ (пережолили / отпустил грудь / вышли из комнаты и т.п.)', 
            'Кол-во ночных проб-й']
columns2 = ['Date', 
            'First wake up time', 
            'Sleep starting time', 
            'Nr of night wake ups']

dict_dc_vis = create_dc(dict_df, columns1, columns2)
                 
for key, value in dict_dc_vis.items():
    print(value.shape)

(967, 4)
(967, 4)
(967, 4)
(967, 4)
(967, 4)


In [ ]:
def create_dc(dc, list1, list2):
    """ Creates a new dictionary with DataFrames from an existing dictionary dc.
    From df1 data from columns from list1 will be copied. The columns in the new DataFrame
    wil be called according to the list2. The order of the names must be the same in list1 and list2.
    """
    dc2 = {}
    for key, value in dc.items():
        dc2[key] = value.loc[:, list1]
        dc2[key].columns = list2
    return dc2

columns1 = ['Дата ', 
            'УТРО (проснулись)', 
            'ВРЕМЯ засыпания в НОЧЬ (пережолили / отпустил грудь / вышли из комнаты и т.п.)', 
            'Кол-во ночных проб-й']

columns3 = ['Date', 
            'First_wake_up_time', 
            'Sleep_starting_time', 
            'Nr_of_night_wake_ups']

dict_df_vis2 = create_dc(dict_df, columns1, columns3)
                 
for key, value in dict_df_vis2.items():
    print(value.shape, value.columns)

In [5]:
for value in dict_dc_vis.values():
    print('Shape before dropna: ', value.shape)
    if value.isnull().values.any():
        value.dropna(axis=0, how='any', inplace=True)
        print('Shape after dropna: ', value.shape)

Shape before dropna:  (967, 4)
Shape after dropna:  (31, 4)
Shape before dropna:  (967, 4)
Shape after dropna:  (49, 4)
Shape before dropna:  (967, 4)
Shape after dropna:  (34, 4)
Shape before dropna:  (967, 4)
Shape after dropna:  (19, 4)
Shape before dropna:  (967, 4)
Shape after dropna:  (21, 4)


In [4]:
for value in dict_df_vis2.values():
    print('Shape before dropna: ', value.shape)
    if value.isnull().values.any():
        value.dropna(axis=0, how='any', inplace=True)
        print('Shape after dropna: ', value.shape)

NameError: name 'dict_df_vis2' is not defined

In [6]:
def cast_type(dc, column, newtype):
    """
    works with dictionary of DataFrames dc
    changes type of the column to newtype
    """
    for value in dc.values():
        value[column] = value[column].astype(newtype)
        
### insert an error for when column not in value.columns

In [9]:
cast_type(dict_dc_vis, 'Nr of night wake ups', int)

for value in dict_dc_vis.values():
    print(value.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 34
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  31 non-null     object
 1   First wake up time    31 non-null     object
 2   Sleep starting time   31 non-null     object
 3   Nr of night wake ups  31 non-null     int32 
dtypes: int32(1), object(3)
memory usage: 1.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 50
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  49 non-null     object
 1   First wake up time    49 non-null     object
 2   Sleep starting time   49 non-null     object
 3   Nr of night wake ups  49 non-null     int32 
dtypes: int32(1), object(3)
memory usage: 1.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 35
Data columns (to

In [ ]:
cast_type(dict_df_vis2, 'Nr_of_night_wake_ups', int)

for value in dict_df_vis2.values():
    print(value.info())

In [ ]:
# alternative filter dor dictionary with dictionary comprehension
# https://blog.finxter.com/how-to-filter-a-dictionary-in-python/
# print({x for x in v for v in dict_dc_vis.values()})

------------------

Experiments with one condition for each comment, different versions

------------------

In [ ]:
### insert a comment to new_column, if condition filter_col applies

new_column = 'New_Column'
filter_col = 'Nr of night wake ups'
comment = 'New test comment'

for value in dict_dc_vis.values():
    value[new_column] = ''
    filter1 = value[filter_col] < 4
    value.loc[filter1, new_column] = comment
    print(value.head())

In [ ]:
## https://stackoverflow.com/questions/42376810/python-dataframes-for-loop-with-if-statement-not-working

def loop(df):
    """
    loop over a DataFrame. 
    If the condition for specified column applies, write comment "Test" to the column "New_Comment"
    """ 
    for index, row in df.iterrows():
        if df.loc[index, 'Nr of night wake ups'] <= 3:
            df.loc[index, 'New_Comment'] = 'Test'
        else:
            df.loc[index, 'New_Comment'] = np.nan
    return (df)
    
def write_comment(df, filter_col, filter_value, comment_col, comment):
    """
    create a new column comment_col
    loop over a DataFrame
    if the condition for column filter_col applies, write comment to the column comment_col
    """ 
    df[comment_col] = np.nan
    for index, row in df.iterrows():
        if df.loc[index, filter_col] <= filter_value:
            df.loc[index, 'New_Comment'] = comment
    return (df)

def write_comment(dc, filter_col, filter_value, comment_col, comment):
    """
    loop over a dictionary of DataFrames
    create a new column comment_col
    loop over a DataFrame
    if the number/string in filter_col is equal filter_value, write comment to the column comment_col
    """  
    for df in dc.values():
        df[comment_col] = np.nan
        for index, row in df.iterrows():
            if df.loc[index, filter_col] == filter_value:
                df.loc[index, comment_col] = comment
    return (dc)

def write_comment(dc, filter_col, function, comment_col, comment):
    """
    loop over a dictionary of DataFrames
    create a new column comment_col
    loop over a DataFrame
    in the comment_col will be written the comment, if condition in function applies, else comment_col = None
    """ 
    for df in dc.values():
        if comment_col not in df.columns:
            df[comment_col] = np.nan
            print('column {} created'.format(comment_col))
        df[comment_col] = df[filter_col].apply(function)
    return (dc)

In [ ]:
### final version of function for writing comments, takes one case from single strings

def write_comment(dc, filter_col, function, comment_col, comment):
    """
    loop over a dictionary of DataFrames
    create a new column comment_col
    loop over a DataFrame
    if the number/string in filter_col is equal filter_value, write comment to the column comment_col
    """ 
    for df in dc.values():
        if comment_col not in df.columns:
            df[comment_col] = np.nan
            print('column {} created'.format(comment_col))
        df.loc[df[filter_col].apply(function), comment_col] = comment
    return (dc)

import numpy as np

filter_col = 'Nr of night wake ups'
function = lambda x: x == 3
new_column = 'Comment'
comment = 'You have only 3 wake ups!'
write_comment(dict_dc_vis, filter_col, function, new_column, comment)

filter_col2 = 'Nr of night wake ups'
function2 = lambda x: x >= 5
new_column2 = 'Comment'
comment2 = "Sorry to hear, let's dig dipper"
write_comment(dict_dc_vis, filter_col2, function2, new_column2, comment2)

filter_col3 = 'Nr of night wake ups'
function3 = lambda x: x >= 5
new_column3 = 'Advice'
comment3 = 'Try changing something'
write_comment(dict_dc_vis, filter_col3, function3, new_column3, comment3)

filter_col4 = 'Nr of night wake ups'
function4 = lambda x: x <=2
new_column4 = 'Comment'
comment4 = 'Congratulations, you have only 2 wake ups!'
write_comment(dict_dc_vis, filter_col4, function4, new_column4, comment4)

print('Comment, comment4 if x <=2 else None')
for value in dict_dc_vis.values():
    print(value.head())

"""
for value in dict_dc_vis.values():
    value[new_column] = np.nan
    value[new_column] = value[filter_col].apply(function)
    print(value.head())
"""

----------------

functions for writing comments with input from lists

----------------

In [ ]:
### function for writing comments with input from lists, writes multiple comments 

def write_multiple_comments(dc, filter_col, function, comment_col, comment):
    """
    loop over a dictionary of DataFrames
    create a new column comment_col
    loop over a DataFrame
    if the number/string in filter_col is equal to the value calculated with function, write comment to comment_col
    """ 
    for i in range(len(comment)):
        for df in dc.values():
            if comment_col[i] not in df.columns:
                df[comment_col[i]] = ''
                #print('column {} created'.format(comment_col[i]))
            df.loc[df[filter_col[i]].apply(function[i]), comment_col[i]] = comment[i]
    return (dc)

import numpy as np

#filter(lambda x: x < 3, data)
filter_col = ['Nr of night wake ups',
              'Nr of night wake ups',
              'Nr of night wake ups',
              'Nr of night wake ups',
              'Nr of night wake ups']
function = [lambda x: x > 3,
            lambda x: x > 3,
            lambda x: x >= 5,
            lambda x: x >= 5,
            lambda x: x <= 3]
new_column = ['Comment',
              'Advice',
              'Comment',
              'Advice',
              'Comment']
comment = ['Верной дорогой идете, товарищи!',
           'Продолжайте в том же духе',
           "Не расстраивайтесь, все будет хорошо",
           "Сделайте глубокий вдох",
           "Вы - мировая мама!"]
           
write_multiple_comments(dict_dc_vis, filter_col, function, new_column, comment)

#for value in dict_dc_vis.values():
#    print(value.head(), '\n')


In [ ]:
"""
## Try out for the write_comment version with an externally specified function and loc in the body of for loop

filter_col = 'Nr of night wake ups'
query = lambda x: (x >= 3) & (x <= 5)
new_column = 'Comment'
comment = 'You have between 3 and 5'
#write_comment(dict_dc_vis, filter_col, function, new_column, comment)

for value in dict_dc_vis.values():
    if new_column not in value.columns:
        value[new_column] = np.nan
    value.loc[value[filter_col].apply(query), new_column] = comment
    
print(value.head())

filter_col2 = 'Nr of night wake ups'
query2 = lambda x: x > 5 
new_column = 'Comment'
comment2 = 'You have more than 5 wake ups!'

for value in dict_dc_vis.values():
    if new_column not in value.columns:
        value[new_column] = np.nan
    value.loc[value[filter_col2].apply(query2), new_column] = comment2
    print(value.head())
    
"""

------

Functions for writing comments for cases, when multiple conditions needed for one comment type

------

In [ ]:
### in the body of for loop it takes the condition from query_list and applies the same filtered dataframe
### over and over again, although the value changes in the for loop

query_list = [(value['Nr of night wake ups'] > 3) & (value['First wake up time'] > '7:10'),
              value['Nr of night wake ups'] == 3,
              value['Nr of night wake ups'] == 3]
column = ['Comment', 
          'Comment', 
          'Advice']

comment = ['Верной дорогой идете, товарищи!',
           "Вы - мировая мама!",
           'Продолжайте в том же духе']


for i in range(len(query_list)):
    print('started with condition {}'.format(i))
    for value in dict_dc_vis.values():
        print(query_list[i])
        print(len(pd.eval(query_list[i])))
        #print(len(value.loc[query_list[i]]))
    #    value.loc[query_list[i], column[i]] = comment[i]
        print('done')


In [ ]:
def w_comment_mult_conditions(df, query_list, comment_col, comment):
    for i in range(len(query_list)):
        df.loc[pd.eval(query_list[i]), comment_col[i]] = comment[i]

query_list = [(value.Nr_of_night_wake_ups > 3) & (value.First_wake_up_time > '7:10'),
              value.Nr_of_night_wake_ups == 3,
              value.Nr_of_night_wake_ups == 3]
comment_col = ['Comment', 
          'Comment', 
          'Advice']
comment = ['Верной дорогой идете, товарищи!',
           "Вы - мировая мама!",
           'Продолжайте в том же духе']

for key, value in dict_df_vis2.items():
    print(pd.eval(query_list[1]))
    #w_comment_mult_conditions(value, query_list, comment_col, comment)
    print('done for ', key)

In [ ]:
for value in dict_dc_vis.values():
    #for i in range(len(query_list)):
        #print(query_list[i], new_column[i], comment[i])
        #if column[i] not in value.columns:
        #    value[column[i]] = ''
    value.loc[(value['Nr of night wake ups'] > 3) & (value['First wake up time'] > '7:10'), 'Comment'] = 'Test'
    print('done')

In [12]:
### works for multiple conditions, but only for one specified comment target column
import numpy as np

"""
conditions = [df['Pclass'].eq(1) & df['Age'].isnull(),
              df['Pclass'].eq(2) & df['Age'].isnull(),
              df['Pclass'].eq(3) & df['Age'].isnull()]

choices = [40,30,25]

df['NewColumn_2'] = np.select(conditions, choices, default= df['Age'] )
"""

filter_conditions = [(value['Nr of night wake ups'] > 3) & (value['First wake up time'] > '7:10'),
              value['Nr of night wake ups'] == 3
              ]

comment = ['Верной дорогой идете, товарищи!',
           "Вы - мировая мама!"
          # 'Продолжайте в том же духе',
           ]

for value in dict_dc_vis.values():
    if 'Comment' not in value.columns:
        value['Comment'] = ''
    value['Comment'] = np.select(filter_conditions, comment, default='')
    print(value.head())

       Date First wake up time Sleep starting time  Nr of night wake ups  \
0  18.01.21               6:10               21:00                     5   
1  19.01.21               8:55               20:50                     3   
2  20.01.21               7:20               21:17                     3   
3  21.01.21               7:20               20:55                     4   
4  22.01.21               7:07               20:55                     3   

                           Comment  
0                                   
1               Вы - мировая мама!  
2               Вы - мировая мама!  
3  Верной дорогой идете, товарищи!  
4               Вы - мировая мама!  


ValueError: Length of values does not match length of index

In [ ]:
### way to write comments, if multiple conditions apply. needs more automatisation

def myfunc(nr, time):
    comment = ''
    if nr == 3:
        comment = "Вы - мировая мама!"
    elif (nr > 3) & (time <= '7:10'):
        comment = 'Верной дорогой идете, товарищи!'
    return comment

#'Nr of night wake ups'] > 3) & (value['First wake up time'
for value in dict_dc_vis.values():
    value['NewColumn_1'] = value.apply(lambda x: myfunc(x['Nr of night wake ups'], x['First wake up time']), axis=1)
    print(value.head())

----------------

create output for generated comments
 
create DataFrame with gspread .create(name_of_spreadsheet) and make it available for your google user
an option to copy comments to the initial dataframe dict_df

----------------

In [ ]:
### write generated comments into DataFrame with initial data

for name in sheet_names:
    dict_df[name]['Автоматизириванные комментарии'] = dict_dc_vis[name]['Comment']
    dict_df[name]['Автоматизированные советы'] = dict_dc_vis[name]['Advice']

for value in dict_df.values():
    print(value.columns)

In [ ]:
### write complete dataframe with two newly generated columns into a new spreadsheet

from gspread_dataframe import set_with_dataframe as set_df

for key, value in dict_df.items():
    worksheet = output_sheet.add_worksheet(title=key, rows=value.shape[0], cols=value.shape[1])
    set_df(worksheet, value, resize=True)

In [ ]:
### Write values from the column 'Advice', converted to a list, to the range in selected
### 1. specify range
### 2. specify column for adv_list

for key, value in dict_dc_vis.items():
    worksheet = sheet.worksheet(key)
    selected = worksheet.range('CT3:CT41') #+ str(len(adv_list) + 3))
    adv_list = value['Advice'].tolist()
    for i, cell in enumerate(selected,0):
        if i < len(adv_list):
            cell.value = adv_list[i]
        else:
            cell.value = ''
        #print(i, cell)
    worksheet.update_cells(selected, value_input_option='USER_ENTERED')
    print('Worksheet "{}" done'.format(key))

Visualisation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

dict_dc_vis_group = {} 
for key, value in dict_dc_vis.items():
    dict_dc_vis_group[key] = value.groupby('Nr of night wake ups', as_index=False).count()
    plt.bar(x='Nr of night wake ups', height='Date', data=dict_dc_vis_group[key])
    plt.xlim(0, 9)
    #plt.xticks(np.arange(0, 8, 1))
    plt.yticks(np.arange(0, 21, 2))
    plt.xlabel('Nr of night wake ups')
    plt.ylabel('Frequency')
    plt.title(key)
    plt.show()